## 1. Scale compute resources in Azure Synapse Analytics

One of the key management features that you have at your disposal within Azure Synapse Analytics, is the ability to scale the compute resources for SQL or Spark pools to meet the demands of processing your data. <b>In SQL pools, the unit of scale is an abstraction of compute power that is known as a data warehouse unit</b>. Compute is separate from storage, which enables you to scale compute independently of the data in your system. This means you can scale up and scale down the compute power to meet your needs.</b>

You can scale a Synapse SQL pool either through the Azure portal, Azure Synapse Studio or programmatically using TSQL or PowerShell.

In [ ]:
## SQL

ALTER DATABASE mySampleDataWarehouse
MODIFY (SERVICE_OBJECTIVE = 'DW300c');

## PowerShell

Set-AzSqlDatabase -ResourceGroupName "resourcegroupname" -DatabaseName "mySampleDataWarehouse" -ServerName "sqlpoolservername" -RequestedServiceObjectiveName "DW300c"

## 2. Scaling Apache Spark pools in Azure Synapse Analytics

Apache Spark pools for Azure Synapse Analytics uses an Autoscale feature that automatically scales the number of nodes in a cluster instance up and down.

## 3. Use dynamic management views to identify and troubleshoot query performance

Dynamic Management Views provide a programmatic experience for monitoring the Azure Synapse Analytics SQL pool activity by using the Transact-SQL language. The views that are provided, not only enable you to troubleshoot and identify performance bottlenecks with the workloads working on your system, but they are also used by other services such as Azure Advisor to provide recommendations about Azure Synapse Analytics

### Monitoring Connections
All logins to your data warehouse are logged to <b>sys.dm_pdw_exec_sessions</b>. The session_id is the primary key and is assigned sequentially for each new logon.

In [ ]:
-- Other Active Connections
SELECT * FROM sys.dm_pdw_exec_sessions where status <> 'Closed' and session_id <> session_id();

### Monitor query execution
All queries executed on SQL pool are logged to <b>sys.dm_pdw_exec_requests</b>. The request_id uniquely identifies each query and is the primary key for this DMV. The request_id is assigned sequentially for each new query and is prefixed with QID, which stands for query ID. Querying this DMV for a given session_id shows all queries for a given logon.

In [ ]:
-- Monitor active queries
SELECT *
FROM sys.dm_pdw_exec_requests
WHERE status not in ('Completed','Failed','Cancelled')
  AND session_id <> session_id()
ORDER BY submit_time DESC;

-- Find top 10 queries longest running queries
SELECT TOP 10 *
FROM sys.dm_pdw_exec_requests
ORDER BY total_elapsed_time DESC;

Queries in the Suspended state can be queued due to a large number of active running queries. These queries also appear in the <b>sys.dm_pdw_waits</b> waits query with a type of UserConcurrencyResourceType.

Use the Request ID to retrieve the queries distributed SQL (DSQL) plan from <b>sys.dm_pdw_request_steps</b>

Use the Request ID and the Step Index to retrieve details from <b>sys.dm_pdw_sql_requests</b>, which contains execution information of the query step on all of the distributed databases.

When the query step is running, <b>DBCC PDW_SHOWEXECUTIONPLAN</b> can be used to retrieve the SQL Server estimated plan from the SQL Server plan cache for the step running on a particular distribution.

In [ ]:
## Find the SQL Server execution plan for a query running on a specific SQL pool or control node.
## Replace distribution_id and spid with values from previous query.

DBCC PDW_SHOWEXECUTIONPLAN(1, 78);

In [ ]:
## Find information about all the workers completing a Data Movement Step.
## Replace request_id and step_index with values from Step 1 and 3.

SELECT * FROM sys.dm_pdw_dms_workers
WHERE request_id = 'QID####' AND step_index = 2;